### Goals
- Convert CSS to SASS:
    - Common values go into variables (e.g. colours)
    - Common groups of styles (>= 2 styles) go into mixins
- NLP to identify similar classes and ids
- NLP to group selectors together and put them into seperate files
- Minify

In [1]:
import os
import shutil
import re
import json
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import rcssmin

In [5]:
with open("minified-dataset\\GlobalTVBC\\css\\23579edb1e12f906d2b83f522d1c27011dc1b1d8.css", "rb") as file:
    css_text_test = file.read().decode(errors="replace")

It appears that after the regular expression styles inside media queries are current extracted except for the first one

In [22]:
styles = re.findall(r'\{\s*(.*?)\s*\}', css_text_test, flags=re.DOTALL) # list of substrings between { and }styles
styles

['position:relative;zoom:1;clear:both;padding-top:10px;margin-bottom:2em;position:relative;zoom:1;width:100%;text-align:left',
 "font-family:'Gotham A','Gotham B',Arial,Helvetica,sans-serif",
 "font-family:'Gotham SSm A','Gotham SSm B',Arial,Helvetica,sans-serif",
 'padding-left:20px;padding-right:20px;width:auto;font-size:76%',
 'position:relative;width:auto;padding:0',
 'margin:5px 0 0 0;padding:0 120px 5px 0;background:none;border:none;text-indent:0;font-size:140%;color:#333',
 'font-weight:normal;display:inline-block;padding-left:5px;font-size:84%',
 'color:#a8a8a8;position:absolute;top:5px;right:10px;text-decoration:none;font-weight:normal;font-size:92%',
 'text-decoration:underline',
 "content:'>> '",
 'padding-right:0',
 'display:none',
 'margin:0;padding:0;background:#dedede;width:100%;height:27px;overflow:hidden',
 'margin:0;padding:0;list-style:none;width:100%;height:27px;font-size:110%',
 "margin:0;padding:0;list-style:none;border-right:1px solid #fff;background:#aaa url('ht

In [23]:
# selector_to_style = {}
# selector = ""
# style = ""
# isInSelector = False
# isInAtRule = False

# for char in css_text_test:
#     if char == "@":
#         isInAtRule = True
#         continue
    
#     if isInAtRule and char == "{":
#         isInAtRule = False
#     elif isInAtRule:
#         continue
#     else:
#         if not isInAtRule and char == "{":
#             isInSelector = True
#             continue

#         if isInSelector:
#             style = f"{style}{char}"

In [28]:
styles = np.array(css_text_test.split("{"))
selectorsInMediaQueries = {}

styles = list(map(lambda selectorStyle: selectorStyle.split("}"), styles))
styles = [j for sub in styles for j in sub] # empty string means end of an at rule

styles

['.resultsBanner',
 'position:relative;zoom:1;clear:both;padding-top:10px;margin-bottom:2em;position:relative;zoom:1;width:100%;text-align:left',
 '.resultsBanner',
 "font-family:'Gotham A','Gotham B',Arial,Helvetica,sans-serif",
 '.resultsBanner',
 "font-family:'Gotham SSm A','Gotham SSm B',Arial,Helvetica,sans-serif",
 '.contain .resultsBanner',
 'padding-left:20px;padding-right:20px;width:auto;font-size:76%',
 '.resultsBanner .hd,.resultsBanner .bd',
 'position:relative;width:auto;padding:0',
 '.resultsBanner .hd h4',
 'margin:5px 0 0 0;padding:0 120px 5px 0;background:none;border:none;text-indent:0;font-size:140%;color:#333',
 '.resultsBanner .hd h4 span.status',
 'font-weight:normal;display:inline-block;padding-left:5px;font-size:84%',
 '.resultsBanner .hd a',
 'color:#a8a8a8;position:absolute;top:5px;right:10px;text-decoration:none;font-weight:normal;font-size:92%',
 '.resultsBanner .hd a:hover',
 'text-decoration:underline',
 '.resultsBanner .hd a:before',
 "content:'>> '",
 '.r

In [ ]:
mediaQuery = cssComponent if "@media" in cssComponent else ""
atRule = cssComponent if "@" in atRule else ""

mediaQuery = "" if cssComponent == "" else mediaQuery
atRule = "" if cssComponent == "" else atRule

In [32]:
selectorToStyle = {}
mediaQuery = ""
atRule = ""
isSelector = True

for i, cssComponent in enumerate(styles):
    if "@media" in cssComponent:
        mediaQuery = cssComponent
        continue
    elif "@" in atRule:
        atRule = cssComponent
        continue

    if isSelector:
        selectorToStyle[cssComponent] = None
        isSelector = False
    else:
        selectorToStyle[styles[i-1]] = {
            "style": cssComponent,
            "atRule": atRule,
            "mediaQuery": mediaQuery
        }
        isSelector = True

selectorToStyle

{'.resultsBanner': {'style': "font-family:'Gotham SSm A','Gotham SSm B',Arial,Helvetica,sans-serif",
  'atRule': '',
  'mediaQuery': ''},
 '.contain .resultsBanner': {'style': 'padding-left:20px;padding-right:20px;width:auto;font-size:76%',
  'atRule': '',
  'mediaQuery': ''},
 '.resultsBanner .hd,.resultsBanner .bd': {'style': 'position:relative;width:auto;padding:0',
  'atRule': '',
  'mediaQuery': ''},
 '.resultsBanner .hd h4': {'style': 'margin:5px 0 0 0;padding:0 120px 5px 0;background:none;border:none;text-indent:0;font-size:140%;color:#333',
  'atRule': '',
  'mediaQuery': ''},
 '.resultsBanner .hd h4 span.status': {'style': 'font-weight:normal;display:inline-block;padding-left:5px;font-size:84%',
  'atRule': '',
  'mediaQuery': ''},
 '.resultsBanner .hd a': {'style': 'color:#a8a8a8;position:absolute;top:5px;right:10px;text-decoration:none;font-weight:normal;font-size:92%',
  'atRule': '',
  'mediaQuery': ''},
 '.resultsBanner .hd a:hover': {'style': 'text-decoration:underline',